In [131]:
import json



In [132]:
def clean_subclasses(subclasses_list):
    #print(subclasses_list)
    data=subclasses_list.split(" | ")
    clean=[]
    if len(data)>1:
        for tr in data:
            spo = tr.split(" - ")
            if spo[0] == spo[2] or spo[2]=='Thing':
                continue
            clean.append((" - ").join(spo))
        return (" | ").join(clean)
    else: return subclasses_list



In [136]:
splits=['train','dev','test']
for split in splits:

    with open(f'/home/gabhoo/research/kg2Narrative/KGNarrative2/Datasets/WebNLG/57_triples/oneClass/Trattini/oneClass_{split}.json', 'r') as f:
        data = json.load(f)

    with open(f'/home/gabhoo/research/kg2Narrative/KGNarrative2/Datasets/WebNLG/57_triples/Multiple_Classes/{split}_57_MultipleClass.json', 'r') as f:
        data_subclass = json.load(f)
    data_subclass=data_subclass[1:] #THIS is because the first line got lost somehow


    instances = ['Instances_KG']

    typeKG = ['Instances_KG', 'Types_KG']

    subClassKG = ['Instances_KG', 'Types_KG','Subclasses_KG']

    for d,d_s in zip(data,data_subclass):
        
        #print("[CORE] "+ d['core_description'] +" [TRIPLES]")

        #MERGE CGRAPHS AND ADD CORE

        merged_types = "[CORE] "+ d['core_description'] +" [TRIPLES] "+' | '.join([d[k] for k in typeKG])

        merged_subClasse = "[CORE] "+d['core_description'] + " [TRIPLES] " + ' | '.join([d[k] for k in subClassKG])

        merged_instances = "[CORE] "+d['core_description'] + " [TRIPLES] " + ' | '.join([d[k] for k in instances])

        d_s['Subclasses_KG']=clean_subclasses(d_s['Subclasses_KG'])
        d['multi_Subclasses_KG'] = "[CORE] "+ d['core_description'] +" [TRIPLES] "+' | '.join([d[k] for k in typeKG]) + d_s['Subclasses_KG']


        d['Types_KG'] = merged_types.replace('"""','')
        d['Subclasses_KG'] = merged_subClasse.replace('"""','')
        d['Instances_KG'] = merged_instances.replace('"""','')

        #ADD CORE TO ENTITIES LIST AND SEMANTIC OF NEWS

        d['Instances_list'] = ("[CORE] "+d['core_description']+ " [ENTITIES] " + " | ".join([x.strip() for x in d['Instances_list']])).replace('"""','')
        d['entities_list'] = ("[CORE] "+d['core_description']+ " [ENTITIES] " + " | ".join([x.strip() for x in d['entities_list']])).replace('"""','')

        d['semantic_of_news'] = ("[CORE] "+d['core_description']+ " [TRIPLES] " + d['semantic_of_news']).replace('"""','')


        
    # Save the updated list of dictionaries back to the same JSON file
    with open(f'/home/gabhoo/research/kg2Narrative/KGNarrative2/Datasets/WebNLG/4experiment/full_{split}.json', 'w') as f:
                json.dump(data,f,indent=4,ensure_ascii = False)
